# Houston, TX Simulations
8760 rows per simulation<br>
50 files (removed cell-in-cell)

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
import pprint
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

foot-candles --> Lux (multiply by 10.7642)

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

In [2]:
# 2018 Egrid Costs (elec $/kWh, gas $/therm), change values per city
gas = 1.11
elec = 0.112

---

## Combine CSV Files in Directory

In [3]:
# Create variable for files in directory
files = [f for f in os.listdir("data/hou/") if f.endswith(".csv")]

# files

In [4]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_data = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/hou/" + file)
    all_data = pd.concat([all_data, df])
    
#all_data

# 438000  rows, 73 columns

In [5]:
# all_data.dtypes

In [6]:
# Get wanted columns
all_data1 = all_data[["Scenario",
               "Date/Time",
              "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
              "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)",
              "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)",
              "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) "]]

#all_data1

In [7]:
all_data1 = all_data1.dropna().reset_index(drop=True)

#all_data1

#438000  rows, 73 columns

-----

## Get the Wanted Data (columns)

In [8]:
# Rename Columns
all_data2 = all_data1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#all_data2

#438000  rows, 6 columns

-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>
Drop Illuminance column<br>


In [9]:
# Copy dataframe to modify and leave original df intact
get_annual = all_data2.copy(deep=True)

#get_annual

In [10]:
# Convert Joules to KBtu in dataframe
get_annual["FanEnergy(kBtu)"] = get_annual["FanEnergy[J](Hourly)"] * .00000094781712
get_annual["HeatingEnergy(kBtu)"] = get_annual["HeatingEnergy[J](Hourly)"] * .00000094781712
get_annual["CoolingEnergy(kBtu)"] = get_annual["CoolingEnergy[J](Hourly) "] * .00000094781712

#get_annual

In [11]:
#list(get_annual.columns.values)

In [12]:
# Drop columns
get_annual.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual

---

## Get Annual Cost Data
Convert units from Joules to kWh and therms to use cost multiplier<br>
Drop Illuminance column<br>
Add cost multiplier, variables defined above

In [13]:
# Get annual costs
get_annual_costs = all_data2.copy(deep=True)

In [14]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual_costs["FanCost($)"] = (get_annual_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_annual_costs["HeatingCost($)"] = ((get_annual_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_annual_costs["CoolingCost($)"] = (get_annual_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_annual_costs

In [15]:
# Drop columns
get_annual_costs.drop(columns = ["Illuminance(lux)","FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual_costs

---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Export combine file

In [16]:
# Combine data for energy and cost
allAnnualData = pd.merge(get_annual, get_annual_costs)

#allAnnualData

In [17]:
# Add energy and cost totals
totalEnergy = allAnnualData["FanEnergy(kBtu)"] + allAnnualData["HeatingEnergy(kBtu)"] + allAnnualData["CoolingEnergy(kBtu)"]
allAnnualData["AnnualEnergy(kBtu)"] = totalEnergy

totalCost = allAnnualData["FanCost($)"] + allAnnualData["HeatingCost($)"] + allAnnualData["CoolingCost($)"]
allAnnualData["AnnualCost($)"] = totalCost

#allAnnualData

In [18]:
#list(allAnnualData.columns.values)

---

## Establish Annual Baseline at Clear/Clear-VB0

Create dataset with just baselines.<br> 
Keep the units in kBtus to be able to eventually combine heating and cooling <br>
Merge the baseline dataset with the full dataset.<br>
Determine the delta between scenario and base.

In [19]:
# Define the scenario for the baselines.
# % improvement from Clear/Clear-VB0 (dual clear with a Venetian Blind at 0 degrees)
baselineAnnual = allAnnualData[allAnnualData["Scenario"] == "Clear/Clear-VB0"]

#baselineAnnual

In [20]:
# Rename columns to indicate baseline to prepare for merge

allBaselineAnnual = baselineAnnual.rename(columns={"Scenario":"B_Scenario",
                              "Date_Time":"Date_Time",
                             "Illuminance(lux)":"B_Illuminance(lux)",
                             "FanEnergy(kBtu)":"B_FanEnergy(kBtu)",
                             "HeatingEnergy(kBtu)":"B_HeatingEnergy(kBtu)",
                             "CoolingEnergy(kBtu)":"B_CoolingEnergy(kBtu)",
                             "AnnualEnergy(kBtu)":"B_AnnualEnergy(kBtu)",
                             "FanCost($)":"B_FanEnergy($)",
                             "HeatingCost($)":"B_HeatingCost($)",
                             "CoolingCost($)":"B_CoolingCost($)",
                             "AnnualCost($)":"B_AnnualCost($)",
                             })

#allBaselineHourly

In [21]:
# Merge the dataframes to include the baseline data.
AnnualWithBaseline = pd.merge(allAnnualData, allBaselineAnnual, on="Date_Time", how="left")

#hourlyWithBaseline

In [22]:
#list(AnnualWithBaseline.columns.values)

In [23]:
# Add column for the difference from baseline columns (scenario - baseline)
AnnualWithBaseline["deltaEnergy(kBtu)"] = AnnualWithBaseline["AnnualEnergy(kBtu)"] - AnnualWithBaseline["B_AnnualEnergy(kBtu)"]
AnnualWithBaseline["deltaCost($)"] = AnnualWithBaseline["AnnualCost($)"] - AnnualWithBaseline["B_AnnualCost($)"]

# AnnualWithBaseline

In [24]:
# Convert NANs to 0
AnnualWithBaseline =AnnualWithBaseline.fillna(0)

#AnnualWithBaseline

---
## Group and Export Annual Data
Group dataframe by scenario<br>
Export to file

In [25]:
# Group by scenario
annualData = AnnualWithBaseline.groupby(["Scenario"], as_index=True)

#annualData.sum().round(2)

In [26]:
annualDataFinal = annualData.sum().round(2)

#annualDataFinal

In [27]:
# Export grouped file to csv
annualDataFinal.to_csv("../Scraper_Output/hou_annual_data.csv", header=True, index=True)

-----

## Get Monthly Energy Data
Change units from Joules to therms and kWh<br>
No baseline data used in monthly export due to differing units of energy, refer to annual and hourly.<br>
Drop unwanted columns

In [28]:
# Copy dataframe to modify and leave original df intact
get_monthly = all_data2.copy(deep=True)

#get_monthly

In [29]:
# Convert Joules to kWh and therms in new dataframe
get_monthly["FanEnergy(kWh)"] = get_monthly["FanEnergy[J](Hourly)"] * .000000277777778
get_monthly["HeatingEnergy(therms)"] = get_monthly["HeatingEnergy[J](Hourly)"] * .0000000094781712
get_monthly["CoolingEnergy(kWh)"] = get_monthly["CoolingEnergy[J](Hourly) "] * .000000277777778

#get_monthly

In [30]:
# Drop columns
get_monthly.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_monthly

## Get Monthly Cost Data

Change units from Joules to therms and kWh<br>
Add cost multiplier, variable needs to be defined above

In [31]:
# Copy dataframe to modify and leave original df intact
get_monthly_costs = all_data2.copy(deep=True)

#get_monthly

In [32]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_monthly_costs["FanCost($)"] = (get_monthly_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_monthly_costs["HeatingCost($)"] = ((get_monthly_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_monthly_costs["CoolingCost($)"] = (get_monthly_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_monthly_costs

In [33]:
# Drop columns
get_monthly_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_monthly_costs

---

## Combine Monthly Energy and Cost Data
Combine two dfs<br>
Create Month Column<br>
Export combine file

In [34]:
# Combine data for energy and cost
allMonthlyData = pd.merge(get_monthly, get_monthly_costs, how="inner")

#allMonthlyData

In [35]:
# Add cost totals, don't total energy...in different units
totalCostMonthly = allMonthlyData["FanCost($)"] + allMonthlyData["HeatingCost($)"] + allMonthlyData["CoolingCost($)"]
allMonthlyData["MonthlyCost($)"] = totalCostMonthly

#allMonthlyData

In [36]:
allMonthlyData["Month"] = allMonthlyData.Date_Time.str.slice(0,3)

#allMonthlyData

## Group and Export Monthly Data
Group dataframe by scenario and month<br>
Export to file

In [37]:
# Group by scenario and date/time for export to graphing
MonthlyData2 = allMonthlyData.groupby(["Scenario", "Month"], as_index=True)

#MonthlyData2.sum()

In [38]:
MonthlyDataFinal = MonthlyData2.sum()

#MonthlyDataFinal

In [39]:
# Drop Illuminance column...don't need the totals
MonthlyDataFinal.drop(columns = ["Illuminance(lux)"], axis=1, inplace=True)

In [40]:
# Export grouped file to csv (used before grouped by month)
MonthlyDataFinal.round(2).to_csv("../Scraper_Output/hou_monthly_data.csv", header=True, index=True)

## Get Monthly Average Illuminance
Get just the illuminance column<br>
Group by scenario <br>
Groups by month <br>

In [41]:
Illuminance = allMonthlyData[["Scenario", "Date_Time", "Illuminance(lux)", "Month"]]

#Illuminance

In [42]:
# Group by scenario
MonthlyIlluminance = Illuminance.groupby(["Scenario", "Month"])

#MonthlyIlluminance.mean()

In [43]:
# Export grouped file to csv
MonthlyIlluminance.mean().round(2).to_csv("../Scraper_Output/hou_monthly_illuminance.csv", header=True, index=True)

------

## Get Hourly Energy Data
Change units from Joules to kBtu to be able to combine heat, cool and fan<br>
Drop unwanted columns

In [44]:
# Copy dataframe to modify and leave original df intact
all_data3 = all_data1.copy(deep=True)

#all_data3

In [45]:
# Get wanted columns
# Include hourly illumance
all_data3 = all_data3[["Scenario",
               "Date/Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) "
                      ]]

#all_data3

In [46]:
# Rename Columns
get_hourly = all_data3.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#get_hourly

In [47]:
# Convert Joules to kBtus in new dataframe
get_hourly["FanEnergy(kBtu)"] = get_hourly["FanEnergy[J](Hourly)"] * .00000094781712
get_hourly["HeatingEnergy(kBtu)"] = get_hourly["HeatingEnergy[J](Hourly)"] * .00000094781712
get_hourly["CoolingEnergy(kBtu)"] = get_hourly["CoolingEnergy[J](Hourly) "] * .00000094781712

#get_hourly

In [48]:
# Drop columns
get_hourly.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_hourly

In [49]:
#list(get_hourly.columns.values)

------

## Get Hourly Cost Data
Add cost multiplier, variable needs to be defined above

In [50]:
# Copy dataframe to modify and leave original df intact
get_hourly_costs = all_data3.copy(deep=True)

#get_hourly_costs

In [51]:
# Rename Columns
get_hourly_costs = get_hourly_costs.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

# get_hourly_costs

In [52]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_hourly_costs["FanCost($)"] = (get_hourly_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_hourly_costs["HeatingCost($)"] = ((get_hourly_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_hourly_costs["CoolingCost($)"] = (get_hourly_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_annual_costs

In [53]:
# Drop columns
get_hourly_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_hourly_costs

## Combine Hourly Energy and Cost Data
Combine two dfs<br>
Export combine file

In [54]:
# Combine data for energy and cost
allHourlyData = pd.merge(get_hourly, get_hourly_costs, how="inner")

#allHourlyData

In [55]:
# Add energy and cost totals

totalEnergyhourly = allHourlyData["FanEnergy(kBtu)"] + allHourlyData["HeatingEnergy(kBtu)"] + allHourlyData["CoolingEnergy(kBtu)"]
totalCosthourly = allHourlyData["FanCost($)"] + allHourlyData["HeatingCost($)"] + allHourlyData["CoolingCost($)"]
allHourlyData["HourlyEnergy(kBtu)"] = totalEnergyhourly
allHourlyData["HourlyCost($)"] = totalCosthourly

#allHourlyData

In [56]:
#list(allHourlyData.columns.values)

---

## Establish Hourly Baseline at Clear/Clear-VB0

Create dataset with just baselines.<br> 
Keep the units in kBtus to be able to eventually combine heating and cooling <br>
Merge the baseline dataset with the full dataset.<br>
Filter the data on the conditionals of illuminance and energy performance.

In [57]:
# Define the scenario for the baselines.
# % improvement from Clear/Clear-VB0 (dual clear with a Venetian Blind at 0 degrees)
baselineHourly = allHourlyData[allHourlyData["Scenario"] == "Clear/Clear-VB0"]

#baselineHourly

In [58]:
# Rename columns to indicate baseline to prepare for merge

allBaselineHourly = baselineHourly.rename(columns={"Scenario":"B_Scenario",
                              "Date_Time":"Date_Time",
                             "Illuminance(lux)":"B_Illuminance(lux)",
                             "FanEnergy(kBtu)":"B_FanEnergy(kBtu)",
                             "HeatingEnergy(kBtu)":"B_HeatingEnergy(kBtu)",
                             "CoolingEnergy(kBtu)":"B_CoolingEnergy(kBtu)",
                             "HourlyEnergy(kBtu)":"B_HourlyEnergy(kBtu)",
                             "FanCost($)":"B_FanEnergy($)",
                             "HeatingCost($)":"B_HeatingCost($)",
                             "CoolingCost($)":"B_CoolingCost($)",
                             "HourlyCost($)":"B_HourlyCost($)",
                             })

#allBaselineHourly

In [59]:
# Merge the dataframes to include the baseline data.
hourlyWithBaseline = pd.merge(allHourlyData, allBaselineHourly, on="Date_Time", how="left")

#hourlyWithBaseline

In [60]:
#list(hourlyWithBaseline.columns.values)

In [61]:
# Add column for the difference from baseline columns (scenario - baseline)
hourlyWithBaseline["deltaEnergy(kBtu)"] = hourlyWithBaseline["HourlyEnergy(kBtu)"] - hourlyWithBaseline["B_HourlyEnergy(kBtu)"]
hourlyWithBaseline["deltaCost($)"] = hourlyWithBaseline["HourlyCost($)"] - hourlyWithBaseline["B_HourlyCost($)"]
hourlyWithBaseline["%deltaEnergy(kBtu)"] = (hourlyWithBaseline["HourlyEnergy(kBtu)"] - hourlyWithBaseline["B_HourlyEnergy(kBtu)"]) / hourlyWithBaseline["B_HourlyEnergy(kBtu)"].replace({0:np.nan}) 
hourlyWithBaseline["%deltaCost($)"] = (hourlyWithBaseline["HourlyCost($)"] - hourlyWithBaseline["B_HourlyCost($)"]) / hourlyWithBaseline["B_HourlyCost($)"].replace({0:np.nan}) 

#hourlyWithBaseline

In [62]:
# Convert NANs to 0
hourlyWithBaseline =hourlyWithBaseline.fillna(0)

#hourlyWithBaseline

In [63]:
# Export hourly data with baseline difference file to csv
hourlyWithBaseline.to_csv("../Scraper_Output/hou_hourly_data.csv", header=True, index=True)